In [6]:
import torch.nn as nn
import gradio as gr
import requests
import pandas as pd
import torch
from transformers import BertTokenizer
from transformers import BertForSequenceClassification

In [5]:
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 642.5 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 1.5 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.4/427.4 kB 1.8 MB/s eta 0:00:0000:0100:01


In [7]:
tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-turkish-128k-uncased")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained(
    "dbmdz/bert-base-turkish-128k-uncased",
    num_labels = 2, 
    output_attentions = False,
    output_hidden_states = False,
).to(device)
model.load_state_dict(torch.load("turkish_fake_news_detection.pth", map_location=device))
model.eval()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-128k-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(128000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [25]:
API_KEY = '550200ce3eae4e808f1924e695de35ad'
API_URL = 'https://newsapi.org/v2/everything'

def extract_data(query):
    params = {
        'q': query,
        'apiKey': API_KEY,
    }
    response = requests.get(API_URL, params=params)
    if response.status_code == 200:
        data = response.json()
        articles_df = pd.DataFrame(data.get('articles', []))
        return articles_df
    else:
        print("Error: Unable to fetch data. Status code:", response.status_code)
        return pd.DataFrame()
    
def predict_verdict(text, model, tokenizer, device, max_length=200):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)
        return "DOĞRU" if preds.item() == 1 else "SAHTE"


def interface(query):
    prediction = predict_verdict(query, model, tokenizer, device)
    return prediction 


iface = gr.Interface(
    fn=interface,
    inputs=[
        "text",
    ],
    outputs="text")

iface.launch(server_port=7880, server_name="0.0.0.0")


Running on local URL:  http://0.0.0.0:7880

To create a public link, set `share=True` in `launch()`.
